# Minimum Code for Training the Model with Google Colab

In [1]:
# Enter the path to the src folder on your drive here.
from google.colab import drive
drive.mount('/content/drive/')
%cd "drive/MyDrive/Module/05 Deep Learning/DeepLearning-FacialLandmarkDetection/src/"

# Alternatively clone the repository.
# !git clone https://github.com/StrangeGirlMurph/Facial-Landmark-Detection.git
# %cd /content/Facial-Landmark-Detection/src
# !git lfs pull

In [ ]:
# If you're curious which GPU you're using :)
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

!nvidia-smi -L
!curl ipinfo.io

## Training

In [ ]:
from util.dataUtil import loadData
from util.modelUtil import saveModel
from modelStructure import defineModel
from training import trainModel

X_train, y_train, X_test = loadData(includeAugmented=True)
model = defineModel()

trainModel(
    model, X_train, y_train,
    epochs=100,
    batch_size=256,
    validation_split=0.2,
    showHistory=True
)

saveModel(model, "../models/modelV4")

## Testing

In [ ]:
from testing import testOnDataset, testOnVideoFile, testOnWebcam
from util.dataUtil import loadData
from util.modelUtil import loadModel

X_train, y_train = loadData(includeTestData=False)
numberOfPoints, seed = 20, 30

model = loadModel("../models/modelV4")
testOnDataset(model, X_train[seed:seed+numberOfPoints], trueValues=y_train[seed:seed+numberOfPoints], show=True, save=False, filename="V4.png")

## Evaluating

In [ ]:
from util.dataUtil import loadData, loadVideoTestData
from util.modelUtil import loadModel
from training import masked_accuracy, masked_mean_absolute_error, masked_mean_squared_error

# Loading the data
X_test, y_test = loadData(includeAugmented=True, percentageOfUncleanData=1)
X_videoTest, y_videoTest = loadVideoTestData()
print("Number of images from the dataset:", len(X_test))
print("Number of video frames:", len(X_videoTest))

# Loading the model
model = loadModel("../models/modelV4")

# Recompile the model to use the custom metrics
model.compile(
    loss=masked_mean_squared_error,
    metrics=[masked_mean_absolute_error, masked_accuracy],
    run_eagerly=True
)

# Evaluating the model on the test data (batches of 32)
print("Results evaluation on the dataset:", model.evaluate(x=X_test, y=y_test))
print("Results evaluation on video frames:", model.evaluate(x=X_videoTest, y=y_videoTest))